# Creating a Chess AI using Tensorflow

In [127]:
import chess
import chess.engine
import numpy as np
import random
from tensorflow.keras import layers, models
import tensorflow as tf

In [133]:
def create_random_board(max_depth = 200):
    """
    Generates board state after a random number of moves in range to max_depth
    If checkmate is reached, board state of checkmate is returned
    Returns board object
    """
    depth = random.randrange(0, max_depth)
    board = chess.Board()
    for _ in range(depth):
        move = random.choice(list(board.legal_moves))
        board.push(move)
        if board.is_game_over():
            break
    return board

def create_dataset(n = 10000, data_type = 'train'):
    """
    Generates a dataset containing n boards, and stockfish's evaluation of those boards.
    Saves boards and corresponding evaluations into two separate numpy files
    """
    sf = chess.engine.SimpleEngine.popen_uci("stockfish")
    positions = []
    evals = []
    for _ in range(n):
        board = create_random_board()
        analysis = sf.analyse(board, chess.engine.Limit(depth=0))
        evaluation = analysis['score'].white().score()
        if evaluation:
            positions.append(convert_board(board))
            evals.append(evaluation)
    positions = np.asarray(positions, dtype = np.int8)
    evals = np.asarray(evals, dtype = np.int16)
    np.save(data_type+'_positions', positions)
    np.save(data_type+'_evals', evals)

def get_dataset(data_type = 'train'):
    """
    Retreive boards and evaluations from file, normalize evaluations to be between 0 and 1
    """
    positions = np.load(data_type+'_positions.npy', allow_pickle = True)
    positions = tf.transpose(positions, [0, 2, 3, 1])
    evals = np.load(data_type+'_evals.npy')
    evals = np.asarray(evals / abs(evals).max() / 2 + 0.5, dtype = np.float32)
    return positions, evals

def convert_board(board):
    """
    Converts board into a 3d 14x8x8 matrix representing the positions of all pieces of 
    both colours, all valid moves, and all attacked squares
    """
    matrix_board = np.zeros((14, 8, 8), dtype = np.int8)
    for piece in chess.PIECE_TYPES:
        mark_pieces(piece, board, matrix_board, chess.WHITE)
        mark_pieces(piece, board, matrix_board, chess.BLACK)
    
    turn_holder = board.turn
    board.turn = chess.WHITE
    mark_valid_moves(board, matrix_board, chess.WHITE)
    board.turn = chess.BLACK
    mark_valid_moves(board, matrix_board, chess.BLACK)
    board.turn = turn_holder
    return matrix_board

def square_to_coords(square):
    """
    Returns coordinate of square in tuple given square name
    e.g. h7 -> (1, 7)
    """
    letter_index = {
        'a': 0,
        'b': 1, 
        'c': 2,
        'd': 3, 
        'e': 4, 
        'f': 5,
        'g': 6, 
        'h': 7
    }
    return 8 - int(square[1]), letter_index[square[0]]


def mark_pieces(piece, board, matrix_board, colour):
    """
    Marks squares in matrix_board with 1 if piece present on board
    """
    matrix_index = -1 if colour == chess.WHITE else 5
    for square in board.pieces(piece, colour):
        coords = np.unravel_index(square, (8, 8))
        matrix_board[piece + matrix_index][7 - coords[0]][coords[1]] = 1

def mark_valid_moves(board, matrix_board, colour):
    """
    Mark all squares in matrix_board that can be reached by a valid move for colour
    """
    matrix_index = 12 if colour == chess.WHITE else 13
    for move in board.legal_moves:
        i, j = square_to_coords(chess.square_name(move.to_square))
        matrix_board[matrix_index][i][j] = 1

def create_model(num_filters, num_layers):
    """
    Creates and returns CNN model with given number of convolution 
    layers with filter size
    """
    model = models.Sequential()
    model.add(layers.Conv2D(filters = 14, kernel_size = 3, padding='same',
                            activation='relu', input_shape = (8, 8, 14)))
    for _ in range(num_layers):
        model.add(layers.Conv2D(filters = num_filters, kernel_size = 3, padding='same',
                                activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, 'relu'))
    model.add(layers.Dense(1, 'sigmoid'))
    return model

In [124]:
%%time
create_dataset(6000, 'train')
create_dataset(1000, 'test')

CPU times: user 1min 45s, sys: 1.39 s, total: 1min 46s
Wall time: 1min 50s


In [134]:
train_positions, train_evals = get_dataset('train')
test_positions, test_evals = get_dataset('test')

In [136]:
model = create_model(32, 4)
model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['accuracy'])
model.summary()
history = model.fit(train_positions, train_evals, batch_size = 2048, epochs = 100, verbose = 1,
                    validation_data = (test_positions, test_evals))
history

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_55 (Conv2D)           (None, 8, 8, 14)          1778      
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 8, 8, 32)          4064      
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 8, 8, 32)          9248      
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 8, 8, 32)          9248      
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 8, 8, 32)          9248      
_________________________________________________________________
flatten_11 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 64)              

[0 0 0]


In [118]:
board = create_random_board()
board = convert_board(board)
x = []
x.append(board)
x = np.asarray(x, dtype = np.int8)


array([0, 0, 0, 0, 0, 0, 0, 0], dtype=int8)

In [128]:
tf.transpose()

5514